### Notebook for tf.eager()

This notebook demonstrates the code in the lecture notes of Thomas Fischbacher on tf.eager().

In [1]:
import numpy
import tensorflow as tf
from __future__ import absolute_import, division, print_function

print('TF Version:', tf.__version__)

# Enabling eager mode for tensorflow.
# This check makes running this cell (and hence enabling tf-eager)idempotent.
try:
  tf.enable_eager_execution()
  print('TF-Eager mode is enabled.')
except ValueError as exn:
  if tf.executing_eagerly():
    print('TF-Eager mode already was enabled.')
  elif 'must be called at program startup' in exn.args[0]:
    print ('Eager-Mode must be enabled at start-time.\n'
           'Please Restart the Runtime '
           '([Runtime] -> [Restart Runtime] or Ctrl-M).')
  else:
    # Unknown situation, re-raise exception.
    raise
    
tfe = tf.contrib.eager

TF Version: 1.10.1
TF-Eager mode is enabled.


In [2]:
tfe.Variable([1,2,3,4]) #A tf.eager variable holds the values

<tf.Variable 'Variable:0' shape=(4,) dtype=int32, numpy=array([1, 2, 3, 4], dtype=int32)>

In [3]:
## This function works when side_lengths is a sequence, a numpy array, 
## or a TensorFlow eager Variable
def f4(side_lengths): 
    
    #t_a,t_b,t_c,t_d = side_lengths # not working with tf.eager
    t_a = side_lengths[0]
    t_b = side_lengths[1]
    t_c = side_lengths[2]
    t_d = side_lengths[3]
    
    t_ab = t_a * t_b
    t_cd = t_c * t_d
    t_abc = t_ab * t_c
    t_abd = t_ab * t_d
    t_acd = t_a * t_cd
    t_bcd = t_b * t_cd
    t_s = t_abc + t_abd + t_acd + t_bcd 
    ret = 2.0 * t_s
    return ret 

p0_list = [1.0, 2.0, 3.0, 4.0]
p0 = tfe.Variable(p0_list)
print(f4(p0_list), f4(numpy.asarray(p0_list)), f4(p0_list))

100.0 100.0 100.0


In [4]:
## More fancy, shorter alternative version
def f4(abcd):
    return 2.0 * sum(abcd[i] * abcd[j] * abcd[k] 
                     for (i, j, k) in [(0, 1, 2), (0, 1, 3),
                                       (0, 2, 3), (1, 2, 3)])

print(f4(p0_list), f4(numpy.asarray(p0_list)), f4(p0_list))

100.0 100.0 100.0


In [5]:
with tf.GradientTape() as tape:
    hypersurface = f4(p0)

In [6]:
gradients_at_p0 = tape.gradient(hypersurface, [p0])
gradients_at_p0

[<tf.Tensor: id=137, shape=(4,), dtype=float32, numpy=array([52., 38., 28., 22.], dtype=float32)>]

In [7]:
print('Hypersurface:', hypersurface.numpy(),
      'Gradient:', gradients_at_p0[0].numpy())

Hypersurface: 100.0 Gradient: [52. 38. 28. 22.]


In [8]:
p1 = tfe.Variable([1.0, 3.0, 3.0, 4.0])

In [9]:
# tape.gradient(hypersurface, [p1]) #You can't use a tape twice
# Will result in 

# 
# ---------------------------------------------------------------------------
#RuntimeError                              Traceback (most recent call last)
#<ipython-input-9-495be9fd48f9> in <module>()
#----> 1 tape.gradient(hypersurface, [p1]) #You can't use a tape twice
#
#/usr/local/lib/python3.5/dist-packages/tensorflow/python/eager/backprop.py in gradient(self, target, sources, output_gradients)
#    830     """
#    831     if self._tape is None:
#--> 832       raise RuntimeError("GradientTape.gradient can only be called once on "
#    833                          "non-persistent tapes.")
#    834     if self._recording:
#
#RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.
#

In [10]:
tape2 = tf.GradientTape()
with tape2: #<-- here we record on the tape to calculate the grads later on
    hypersurface1 = f4(p0)
    hypersurface2 = f4(2 * p0)
    total_hypersurface = hypersurface1 + hypersurface2
    for i in range(10):
        total_hypersurface += total_hypersurface

#<--- here we can (still) access the tape 

In [11]:
tape2.gradient(total_hypersurface, p0)

<tf.Tensor: id=402, shape=(4,), dtype=float32, numpy=array([479232., 350208., 258048., 202752.], dtype=float32)>